### Résultats de l'assimilation

In [1]:
import psycopg2
import pandas as pd
import re
import os

# Chemin vers le dossier principal contenant les sous-dossiers (villes)
dossier_principal = "dossier_code"

# Dictionnaire pour stocker les informations par ville, y compris les listes
infos_villes = {}

# Parcours des dossiers (villes)
for ville in os.listdir(dossier_principal):
    chemin_ville = os.path.join(dossier_principal, ville)

    # Vérification si l'élément est un dossier
    if os.path.isdir(chemin_ville):
        # Initialisation des listes de codes postaux uniques pour cette ville
        codes_postaux = {
            "15_min": set(),
            "30_min": set(),
            "45_min": set(),
            "total": set()  # Ajout de la liste totale
        }

        for fichier in os.listdir(chemin_ville):
            chemin_fichier = os.path.join(chemin_ville, fichier)

            # Vérification si l'élément est un fichier Excel
            if os.path.isfile(chemin_fichier) and chemin_fichier.endswith(".xlsx"):
                # Lecture du fichier Excel
                df = pd.read_excel(chemin_fichier)

                codes = set(df["Code postal"].tolist())  # Convertir en ensemble (set)

                # Mise à jour des codes postaux uniques par minute
                if fichier == "15.xlsx":
                    codes_postaux["15_min"].update(codes)
                elif fichier == "30.xlsx":
                    codes_postaux["30_min"].update(codes - codes_postaux["15_min"])
                elif fichier == "45.xlsx":
                    codes_postaux["45_min"].update(codes - codes_postaux["30_min"] - codes_postaux["15_min"])

                # Mise à jour de la liste totale
                codes_postaux["total"].update(codes)

        # Stockage des informations par ville, y compris les listes
        nom_dossier = os.path.basename(dossier_principal)
        nom_ville = os.path.basename(chemin_ville)
        infos_villes[nom_ville] = {
            "Nom_Dossier": nom_dossier,
            "Nom_Ville": nom_ville,
            "Codes_Postaux_15_min": len(codes_postaux["15_min"]),
            "Codes_Postaux_30_min": len(codes_postaux["30_min"]),
            "Codes_Postaux_45_min": len(codes_postaux["45_min"]),
            "Codes_Postaux_Uniques": len(codes_postaux["total"]),  # Utilisation de la liste totale pour les codes uniques
            "Codes_Postaux_Listes": codes_postaux  # Ajout des listes de codes postaux
        }

# Affichage des informations par ville et des listes de codes postaux uniques
for nom_ville, info in infos_villes.items():
    print(f"Dossier principal: {info['Nom_Dossier']}")
    print(f"Nom de la ville: {info['Nom_Ville']}")
    print("Nombre de codes postaux uniques de 15 min:", info["Codes_Postaux_15_min"])
    print("Nombre de codes postaux uniques de 30 min:", info["Codes_Postaux_30_min"])
    print("Nombre de codes postaux uniques de 45 min:", info["Codes_Postaux_45_min"])
    print("Nombre de codes postaux uniques au total:", info["Codes_Postaux_Uniques"])
    print("Liste de codes postaux uniques de 15 min:", info["Codes_Postaux_Listes"]["15_min"])
    print("Liste de codes postaux uniques de 30 min:", info["Codes_Postaux_Listes"]["30_min"])
    print("Liste de codes postaux uniques de 45 min:", info["Codes_Postaux_Listes"]["45_min"])
    print("Liste de codes postaux uniques au total:", info["Codes_Postaux_Listes"]["total"])
    print("\n")

# Établir la connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="Base",
    user="waouhmonde",
    password="waouhmonde"
)

for nom_ville, info in infos_villes.items():
    print(f"Traitement de la ville : {nom_ville}")
    for valeur_type in ["15_min", "30_min", "45_min", "total"]:
        print(f"{nom_ville} - {valeur_type}")
        codes_postaux_liste = info["Codes_Postaux_Listes"][valeur_type]
        # Convertissez les codes postaux en chaînes de caractères (str)
        codes_postaux_str = [str(code_postal) for code_postal in codes_postaux_liste]

        # Créez une requête SQL pour rechercher les lignes correspondantes dans votre table
        query = "SELECT * FROM ma_table WHERE code_postal IN %s"

        # Exécutez la requête SQL en utilisant les codes postaux convertis comme paramètres
        df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))

        # Si le DataFrame est vide, affichez un message
        if df.empty:
            print("Aucun résultat trouvé pour cette ville.")
        else:
            # Supprimer les doublons en fonction du numéro de téléphone
            df.drop_duplicates(subset=['numero_telephone'], keep='first', inplace=True)

            def extract_nom_prenom(row):
                nom_prenom = row['nom'].split(' ')
                nom = nom_prenom[0]
                prenom = ' '.join(nom_prenom[1:])
                return pd.Series([nom, prenom], index=['Nom', 'Prenom'])

            # Appliquer la fonction au DataFrame et concaténer le résultat
            df = pd.concat([df, df.apply(extract_nom_prenom, axis=1)], axis=1)

            # Supprimer la colonne 'nom' originale si nécessaire
            df = df.drop('nom', axis=1)

            # Lire le fichier Excel dans df1
            nom_du_fichier = 'prenoms_arabes.xlsx'
            df1 = pd.read_excel(nom_du_fichier)

            prenoms_arabes = df1['Prénoms']

            # Filtrer les prénoms arabes
            masque = df['Prenom'].isin(prenoms_arabes)
            df = df[~masque]

            # Définition des fonctions pour les numéros mobiles et fixes
            def est_mobile(numero):
                motif_mobile = r'^(6\d{8}|\+336\d{8}|7\d{8})'  # Inclure les numéros commençant par "7" comme numéros mobiles
                return bool(re.match(motif_mobile, str(numero)))  # Convertir en chaîne de caractères

            def est_fixe(numero):
                motif_fixe = r'^(0[1-5]\d{7}|0[8-9]\d{7}|[1-5]\d{7}|[6-9]\d{7}|9\d{7})'  # Inclure les numéros commençant par "9" comme numéros fixes
                return bool(re.match(motif_fixe, str(numero)))  # Convertir en chaîne de caractères

            # Filtrer les numéros mobiles et fixes
            df_mobile = df[df['numero_telephone'].apply(est_mobile)].copy()
            df_fixe = df[df['numero_telephone'].apply(est_fixe)].copy()
            
            if valeur_type == "total" :
                nom_fichier_fixe = f"{nom_ville}Fixe.xlsx"
                nom_fichier_mobile = f"{nom_ville}Mobile.xlsx"
                df_fixe.to_excel(nom_fichier_fixe, index=False)
                df_mobile.to_excel(nom_fichier_mobile, index=False)
                print("fichier chargés")
            # Créer une DataFrame pour les numéros inconnus
            df_inconnu = df[~df['numero_telephone'].apply(est_mobile) & ~df['numero_telephone'].apply(est_fixe)].copy()

            # Obtenir une liste des numéros inconnus
            numeros_inconnus = df_inconnu['numero_telephone'].tolist()

            # Compter les lignes où le numéro est vide ou None
            numero_vide_count = df[df['numero_telephone'].isna() | df['numero_telephone'] == ''].shape[0]

            # Nombre de lignes pour les numéros fixes
            nombre_fixe = df_fixe.shape[0]

            # Nombre de lignes pour les numéros mobiles
            nombre_mobile = df_mobile.shape[0]

            # Somme des numéros fixes et mobiles
            somme_fixe_mobile = nombre_fixe + nombre_mobile

            print(f"Nombre de lignes avec des numéros vides ou None : {numero_vide_count}")
            print(f"Nombre de lignes pour les numéros fixes : {nombre_fixe}")
            print(f"Nombre de lignes pour les numéros mobiles : {nombre_mobile}")
            print(f"Somme des numéros fixes et mobiles : {somme_fixe_mobile}")

            print("\n")

# Fermez la connexion à la base de données
conn.close()

Dossier principal: dossier_code
Nom de la ville: AURILLAC
Nombre de codes postaux uniques de 15 min: 4
Nombre de codes postaux uniques de 30 min: 9
Nombre de codes postaux uniques de 45 min: 11
Nombre de codes postaux uniques au total: 24
Liste de codes postaux uniques de 15 min: {15000, 15130, 15220, 15250}
Liste de codes postaux uniques de 30 min: {15140, 15590, 15340, 15310, 15150, 15120, 15600, 15800, 15290}
Liste de codes postaux uniques de 45 min: {12320, 46210, 15300, 19430, 12300, 12460, 12140, 46190, 19220, 15700, 12600}
Liste de codes postaux uniques au total: {46210, 15300, 12300, 15310, 15120, 15250, 19220, 15700, 15000, 15130, 12320, 12600, 15140, 15590, 19430, 15340, 12460, 15150, 12140, 15600, 46190, 15220, 15800, 15290}


Dossier principal: dossier_code
Nom de la ville: CRECHE_SUR_SOANE
Nombre de codes postaux uniques de 15 min: 9
Nombre de codes postaux uniques de 30 min: 37
Nombre de codes postaux uniques de 45 min: 72
Nombre de codes postaux uniques au total: 118
Lis

C:\Users\Admin\AppData\Local\Temp\ipykernel_10256\1373529742.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


Nombre de lignes avec des numéros vides ou None : 0
Nombre de lignes pour les numéros fixes : 17638
Nombre de lignes pour les numéros mobiles : 546
Somme des numéros fixes et mobiles : 18184


AURILLAC - 30_min


C:\Users\Admin\AppData\Local\Temp\ipykernel_10256\1373529742.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


Nombre de lignes avec des numéros vides ou None : 0
Nombre de lignes pour les numéros fixes : 8663
Nombre de lignes pour les numéros mobiles : 301
Somme des numéros fixes et mobiles : 8964


AURILLAC - 45_min


...

Nombre de lignes avec des numéros vides ou None : 0
Nombre de lignes pour les numéros fixes : 34473
Nombre de lignes pour les numéros mobiles : 1569
Somme des numéros fixes et mobiles : 36042


VALENCE - 45_min


C:\Users\Admin\AppData\Local\Temp\ipykernel_10256\1373529742.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


Nombre de lignes avec des numéros vides ou None : 0
Nombre de lignes pour les numéros fixes : 24937
Nombre de lignes pour les numéros mobiles : 1064
Somme des numéros fixes et mobiles : 26001


VALENCE - total


C:\Users\Admin\AppData\Local\Temp\ipykernel_10256\1373529742.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))


fichier chargés
Nombre de lignes avec des numéros vides ou None : 0
Nombre de lignes pour les numéros fixes : 104234
Nombre de lignes pour les numéros mobiles : 4530
Somme des numéros fixes et mobiles : 108764




In [ ]:
import psycopg2
import pandas as pd
import re
import os

# Chemin vers le dossier principal contenant les sous-dossiers (villes)
dossier_principal = "dossier_code"

# Dictionnaire pour stocker les informations par ville, y compris les listes
infos_villes = {}

# Parcours des dossiers (villes)
for ville in os.listdir(dossier_principal):
    chemin_ville = os.path.join(dossier_principal, ville)

    # Vérification si l'élément est un dossier
    if os.path.isdir(chemin_ville):
        # Initialisation des listes de codes postaux uniques pour cette ville
        codes_postaux = {
            "15_min": set(),
            "30_min": set(),
            "45_min": set(),
            "total": set()  # Ajout de la liste totale
        }

        for fichier in os.listdir(chemin_ville):
            chemin_fichier = os.path.join(chemin_ville, fichier)

            # Vérification si l'élément est un fichier Excel
            if os.path.isfile(chemin_fichier) and chemin_fichier.endswith(".xlsx"):
                # Lecture du fichier Excel
                df = pd.read_excel(chemin_fichier)

                codes = set(df["Code postal"].tolist())  # Convertir en ensemble (set)

                # Mise à jour des codes postaux uniques par minute
                if fichier == "15.xlsx":
                    codes_postaux["15_min"].update(codes)
                elif fichier == "30.xlsx":
                    codes_postaux["30_min"].update(codes - codes_postaux["15_min"])
                elif fichier == "45.xlsx":
                    codes_postaux["45_min"].update(codes - codes_postaux["30_min"] - codes_postaux["15_min"])

                # Mise à jour de la liste totale
                codes_postaux["total"].update(codes)

        # Stockage des informations par ville, y compris les listes
        nom_dossier = os.path.basename(dossier_principal)
        nom_ville = os.path.basename(chemin_ville)
        infos_villes[nom_ville] = {
            "Nom_Dossier": nom_dossier,
            "Nom_Ville": nom_ville,
            "Codes_Postaux_15_min": len(codes_postaux["15_min"]),
            "Codes_Postaux_30_min": len(codes_postaux["30_min"]),
            "Codes_Postaux_45_min": len(codes_postaux["45_min"]),
            "Codes_Postaux_Uniques": len(codes_postaux["total"]),  # Utilisation de la liste totale pour les codes uniques
            "Codes_Postaux_Listes": codes_postaux  # Ajout des listes de codes postaux
        }

# Affichage des informations par ville et des listes de codes postaux uniques
for nom_ville, info in infos_villes.items():
    print(f"Dossier principal: {info['Nom_Dossier']}")
    print(f"Nom de la ville: {info['Nom_Ville']}")
    print("Nombre de codes postaux uniques de 15 min:", info["Codes_Postaux_15_min"])
    print("Nombre de codes postaux uniques de 30 min:", info["Codes_Postaux_30_min"])
    print("Nombre de codes postaux uniques de 45 min:", info["Codes_Postaux_45_min"])
    print("Nombre de codes postaux uniques au total:", info["Codes_Postaux_Uniques"])
    print("Liste de codes postaux uniques de 15 min:", info["Codes_Postaux_Listes"]["15_min"])
    print("Liste de codes postaux uniques de 30 min:", info["Codes_Postaux_Listes"]["30_min"])
    print("Liste de codes postaux uniques de 45 min:", info["Codes_Postaux_Listes"]["45_min"])
    print("Liste de codes postaux uniques au total:", info["Codes_Postaux_Listes"]["total"])
    print("\n")

# Établir la connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="Base",
    user="waouhmonde",
    password="waouhmonde"
)

for nom_ville, info in infos_villes.items():
    print(f"Traitement de la ville : {nom_ville}")
    for valeur_type in ["15_min", "30_min", "45_min", "total"]:
        print(f"{nom_ville} - {valeur_type}")
        codes_postaux_liste = info["Codes_Postaux_Listes"][valeur_type]
        # Convertissez les codes postaux en chaînes de caractères (str)
        codes_postaux_str = [str(code_postal) for code_postal in codes_postaux_liste]

        # Créez une requête SQL pour rechercher les lignes correspondantes dans votre table
        query = "SELECT * FROM ma_table WHERE code_postal IN %s"

        # Exécutez la requête SQL en utilisant les codes postaux convertis comme paramètres
        df = pd.read_sql_query(query, conn, params=(tuple(codes_postaux_str),))

        # Si le DataFrame est vide, affichez un message
        if df.empty:
            print("Aucun résultat trouvé pour cette ville.")
        else:
            # Supprimer les doublons en fonction du numéro de téléphone
            df.drop_duplicates(subset=['numero_telephone'], keep='first', inplace=True)

            def extract_nom_prenom(row):
                nom_prenom = row['nom'].split(' ')
                nom = nom_prenom[0]
                prenom = ' '.join(nom_prenom[1:])
                return pd.Series([nom, prenom], index=['Nom', 'Prenom'])

            # Appliquer la fonction au DataFrame et concaténer le résultat
            df = pd.concat([df, df.apply(extract_nom_prenom, axis=1)], axis=1)

            # Supprimer la colonne 'nom' originale si nécessaire
            df = df.drop('nom', axis=1)

            # Lire le fichier Excel dans df1
            nom_du_fichier = 'prenoms_arabes.xlsx'
            df1 = pd.read_excel(nom_du_fichier)

            prenoms_arabes = df1['Prénoms']

            # Filtrer les prénoms arabes
            masque = df['Prenom'].isin(prenoms_arabes)
            df = df[~masque]

            # Définition des fonctions pour les numéros mobiles et fixes
            def est_mobile(numero):
                motif_mobile = r'^(6\d{8}|\+336\d{8}|7\d{8})'  # Inclure les numéros commençant par "7" comme numéros mobiles
                return bool(re.match(motif_mobile, str(numero)))  # Convertir en chaîne de caractères

            def est_fixe(numero):
                motif_fixe = r'^(0[1-5]\d{7}|0[8-9]\d{7}|[1-5]\d{7}|[6-9]\d{7}|9\d{7})'  # Inclure les numéros commençant par "9" comme numéros fixes
                return bool(re.match(motif_fixe, str(numero)))  # Convertir en chaîne de caractères

            # Filtrer les numéros mobiles et fixes
            df_mobile = df[df['numero_telephone'].apply(est_mobile)].copy()
            df_fixe = df[df['numero_telephone'].apply(est_fixe)].copy()
            
            if valeur_type == "total" :
                nom_fichier_fixe = f"{nom_ville}Fixe.xlsx"
                nom_fichier_mobile = f"{nom_ville}Mobile.xlsx"
                df_fixe.to_excel(nom_fichier_fixe, index=False)
                df_mobile.to_excel(nom_fichier_mobile, index=False)
                print("fichier chargés")
            # Créer une DataFrame pour les numéros inconnus
            df_inconnu = df[~df['numero_telephone'].apply(est_mobile) & ~df['numero_telephone'].apply(est_fixe)].copy()

            # Obtenir une liste des numéros inconnus
            numeros_inconnus = df_inconnu['numero_telephone'].tolist()

            # Compter les lignes où le numéro est vide ou None
            numero_vide_count = df[df['numero_telephone'].isna() | df['numero_telephone'] == ''].shape[0]

            # Nombre de lignes pour les numéros fixes
            nombre_fixe = df_fixe.shape[0]

            # Nombre de lignes pour les numéros mobiles
            nombre_mobile = df_mobile.shape[0]

            # Somme des numéros fixes et mobiles
            somme_fixe_mobile = nombre_fixe + nombre_mobile

            print(f"Nombre de lignes avec des numéros vides ou None : {numero_vide_count}")
            print(f"Nombre de lignes pour les numéros fixes : {nombre_fixe}")
            print(f"Nombre de lignes pour les numéros mobiles : {nombre_mobile}")
            print(f"Somme des numéros fixes et mobiles : {somme_fixe_mobile}")

            print("\n")

# Fermez la connexion à la base de données
conn.close()